In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Initialize the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [8]:
from keras.preprocessing import image
import numpy as np

# Load the image
img_path = 'C:/Users/maryc/Desktop/pythonproject/Datacluster Fire and Smoke Sample/training/Datacluster Fire and Smoke Sample (21).jpg'  # replace with the path to your image
img = image.load_img(img_path, target_size=(64, 64))

# Preprocess the image
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.  # the model was trained on inputs that were normalized to [0, 1]

# Use the model to predict the class of the image
prediction = classifier.predict(img_tensor)

# Print the prediction
if prediction > 0.5:
    print("The image is predicted to contain fire.")
else:
    print("The image is predicted to not contain fire.")


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/maryc/Desktop/pythonproject/Datacluster Fire and Smoke Sample/training/Datacluster Fire and Smoke Sample (21).jpg'

In [ ]:
#simplified and less overfitting around 70%
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
import os
import numpy as np
import cv2

# Assuming that data_dir is the directory where your data is stored
data_dir = '../work/training'  # replace with the path to your data

# Get the list of all the images
image_paths = []
labels = []
for class_dir in os.listdir(data_dir):
    class_dir_path = os.path.join(data_dir, class_dir)
    for image_path in os.listdir(class_dir_path):
        image_paths.append(os.path.join(class_dir_path, image_path))
        labels.append(class_dir)

# Convert labels to integers
unique_labels = np.unique(labels)
label_dict = {label: i for i, label in enumerate(unique_labels)}
labels = np.array([label_dict[label] for label in labels])

# One-hot encode labels
labels = to_categorical(labels)

# Convert one-hot encoded labels to single labels
labels_single = np.argmax(labels, axis=1)

# Perform k-fold cross-validation
kfold = StratifiedKFold(n_splits=5, shuffle=True)

for train, test in kfold.split(image_paths, labels_single):
    model = Sequential()

    model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(4, activation='softmax'))

    model.compile(optimizer=Adam(),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Load and preprocess the training images
    train_images = np.array([cv2.resize(cv2.imread(path), (150, 150)) for path in np.array(image_paths)[train]])
    train_images = train_images / 255.0
    train_labels = labels[train]

    # Load and preprocess the test images
    test_images = np.array([cv2.resize(cv2.imread(path), (150, 150)) for path in np.array(image_paths)[test]])
    test_images = test_images / 255.0
    test_labels = labels[test]

    history = model.fit(train_images, train_labels, epochs=25, validation_data=(test_images, test_labels))
    
    score = model.evaluate(test_images, test_labels)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])
